In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:000:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing ins

In [2]:
import numpy as np
import pandas as pd
import torch
import os
import yaml
import cv2
import json
import torch

In [3]:
import json
import pandas as pd

# Загрузка аннотаций
annotations_path = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train/_annotations.coco.json'
with open(annotations_path) as f:
    coco_data = json.load(f)


categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
images = {img['id']: img['file_name'] for img in coco_data['images']}


target_category_ids = {0, 1, 2, 3, 4, 5}
unique_image_ids = set()


data = []
for ann in coco_data['annotations']:
    category_id = ann['category_id']
    if category_id in target_category_ids:
        image_id = ann['image_id']
        file_name = images.get(image_id, f"unknown_{image_id}")
        unique_image_ids.add(file_name)

        x, y, w, h = ann['bbox']
        obj_record = [categories[category_id], x, y, w, h]

        found = False
        for entry in data:
            if entry['image_id'] == file_name:
                entry['objects'].append(obj_record)
                found = True
                break

        if not found:
            data.append({'image_id': file_name, 'objects': [obj_record]})


df = pd.DataFrame(data)
df['objects'] = df['objects'].apply(lambda x: str(x).replace("'", '"'))

# Вывод количества уникальных фотографий
num_unique_images = len(unique_image_ids)
print(f"Количество уникальных фотографий с интересующими категориями: {num_unique_images}")

Количество уникальных фотографий с интересующими категориями: 754


In [4]:
annotations_path = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train/_annotations.coco.json'
with open(annotations_path) as f:
    coco_data = json.load(f)

unique_categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
num_categories = len(unique_categories)

print(f"Количество уникальных категорий: {num_categories}")
print("Список категорий:")
for category_id, category_name in unique_categories.items():
    print(f"ID: {category_id}, Название: {category_name}")

Количество уникальных категорий: 6
Список категорий:
ID: 0, Название: damage
ID: 1, Название: corrosion
ID: 2, Название: lightning
ID: 3, Название: lightning receptor
ID: 4, Название: missing teeth
ID: 5, Название: patch


# Колво фоток по категориям на train

In [5]:
annotations_path = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train/_annotations.coco.json'
with open(annotations_path) as f:
    coco_data = json.load(f)

categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
images = {img['id']: img['file_name'] for img in coco_data['images']}

category_image_counts = {cat_id: set() for cat_id in categories.keys()}

for ann in coco_data['annotations']:
    category_id = ann['category_id']
    image_id = ann['image_id']
    file_name = images.get(image_id, f"unknown_{image_id}")

    if category_id in category_image_counts:
        category_image_counts[category_id].add(file_name)

print("Количество уникальных фотографий по категориям:")
for cat_id, file_names in category_image_counts.items():
    num_unique_images = len(file_names)
    category_name = categories[cat_id]
    print(f"ID: {cat_id}, Название: {category_name}, Количество фотографий: {num_unique_images}")

Количество уникальных фотографий по категориям:
ID: 0, Название: damage, Количество фотографий: 0
ID: 1, Название: corrosion, Количество фотографий: 534
ID: 2, Название: lightning, Количество фотографий: 31
ID: 3, Название: lightning receptor, Количество фотографий: 82
ID: 4, Название: missing teeth, Количество фотографий: 372
ID: 5, Название: patch, Количество фотографий: 76


# Колво фоток по категориям на валидации

In [6]:
annotations_path = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/valid/_annotations.coco.json'
with open(annotations_path) as f:
    coco_data = json.load(f)

categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
images = {img['id']: img['file_name'] for img in coco_data['images']}

category_image_counts = {cat_id: set() for cat_id in categories.keys()}

for ann in coco_data['annotations']:
    category_id = ann['category_id']
    image_id = ann['image_id']
    file_name = images.get(image_id, f"unknown_{image_id}")

    if category_id in category_image_counts:
        category_image_counts[category_id].add(file_name)

print("Количество уникальных фотографий по категориям:")
for cat_id, file_names in category_image_counts.items():
    num_unique_images = len(file_names)
    category_name = categories[cat_id]
    print(f"ID: {cat_id}, Название: {category_name}, Количество фотографий: {num_unique_images}")

Количество уникальных фотографий по категориям:
ID: 0, Название: damage, Количество фотографий: 0
ID: 1, Название: corrosion, Количество фотографий: 134
ID: 2, Название: lightning, Количество фотографий: 9
ID: 3, Название: lightning receptor, Количество фотографий: 10
ID: 4, Название: missing teeth, Количество фотографий: 81
ID: 5, Название: patch, Количество фотографий: 16


# Аугментация

In [8]:
import albumentations as A
from tqdm import tqdm
import ast

In [9]:
annotations_path = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train/_annotations.coco.json'

images_folder = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train'
output_folder = '/kaggle/working/'

os.makedirs(output_folder, exist_ok=True)

with open(annotations_path) as f:
    coco_data = json.load(f)

categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
images = {img['id']: img['file_name'] for img in coco_data['images']}

data = []
for ann in coco_data['annotations']:
    image_id = ann['image_id']
    file_name = images.get(image_id, f"unknown_{image_id}")
    category = categories.get(ann['category_id'], "unknown")
    x, y, w, h = ann['bbox']
    obj_record = [category, x, y, w, h]

    found = False
    for entry in data:
        if entry['image_id'] == file_name:
            entry['objects'].append(obj_record)
            found = True
            break

    if not found:
        data.append({'image_id': file_name, 'objects': [obj_record]})

df = pd.DataFrame(data)
df['objects'] = df['objects'].apply(lambda x: str(x).replace("'", '"'))

transform = A.Compose([
    A.SmallestMaxSize(max_size=640),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.2),
    A.ToFloat(max_value=255.0)  # нормализуем значения пикселей
], bbox_params=A.BboxParams(format='coco', label_fields=['category_names']))

def augment_image(row, augmentations_per_image=4):
    new_rows = []
    image_path = os.path.join(images_folder, row['image_id'])
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    objects = ast.literal_eval(row['objects'])
    bboxes = [obj[1:] for obj in objects]
    category_names = [obj[0] for obj in objects]

    for i in range(augmentations_per_image):
        try:
            transformed = transform(
                image=image,
                bboxes=bboxes,
                category_names=category_names
            )

            new_filename = f"aug{i}_{row['image_id']}"
            cv2.imwrite(os.path.join(output_folder, new_filename),
                        cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR))

            new_objects = [
                [cat, *bbox]
                for bbox, cat in zip(transformed['bboxes'], transformed['category_names'])
            ]

            new_rows.append({
                'image_id': new_filename,
                'objects': str(new_objects).replace("'", '"')
            })
        except Exception as e:
            print(f"Ошибка при аугментации {row['image_id']}: {str(e)}")

    return new_rows

augmented_data = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    augmented_data.append(row.to_dict())
    augmented_data.extend(augment_image(row))

augmented_df = pd.DataFrame(augmented_data)

augmented_df.to_csv(os.path.join(output_folder, 'augmented_annotations.csv'), index=False)

print(f"Оригинальный датасет: {len(df)} изображений")
print(f"Аугментированный датасет: {len(augmented_df)} изображений")
print("\nПример аугментированных данных:")
print(augmented_df.head())

/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:87: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
  6%|▌         | 47/754 [00:04<01:01, 11.46it/s]

Ошибка при аугментации 20240120_162942_jpg.rf.0270798612537561a390df5dbdd72a98.jpg: Expected y_max for bbox [0.6520722 0.967246  0.6862934 1.0001336 0.       ] to be in the range [0.0, 1.0], got 1.0001336336135864.
Ошибка при аугментации 20240120_162942_jpg.rf.0270798612537561a390df5dbdd72a98.jpg: Expected y_max for bbox [0.6520722 0.967246  0.6862934 1.0001336 0.       ] to be in the range [0.0, 1.0], got 1.0001336336135864.
Ошибка при аугментации 20240120_162942_jpg.rf.0270798612537561a390df5dbdd72a98.jpg: Expected y_max for bbox [0.6520722 0.967246  0.6862934 1.0001336 0.       ] to be in the range [0.0, 1.0], got 1.0001336336135864.
Ошибка при аугментации 20240120_162942_jpg.rf.0270798612537561a390df5dbdd72a98.jpg: Expected y_max for bbox [0.6520722 0.967246  0.6862934 1.0001336 0.       ] to be in the range [0.0, 1.0], got 1.0001336336135864.


 20%|██        | 153/754 [00:16<01:02,  9.59it/s]

Ошибка при аугментации 20240120_162912_jpg.rf.29699b9c2c3bc8888f76923fb517f311.jpg: Expected y_max for bbox [0.7616979 0.9906417 0.8364572 1.0000601 0.       ] to be in the range [0.0, 1.0], got 1.0000600814819336.
Ошибка при аугментации 20240120_162912_jpg.rf.29699b9c2c3bc8888f76923fb517f311.jpg: Expected y_max for bbox [0.7616979 0.9906417 0.8364572 1.0000601 0.       ] to be in the range [0.0, 1.0], got 1.0000600814819336.
Ошибка при аугментации 20240120_162912_jpg.rf.29699b9c2c3bc8888f76923fb517f311.jpg: Expected y_max for bbox [0.7616979 0.9906417 0.8364572 1.0000601 0.       ] to be in the range [0.0, 1.0], got 1.0000600814819336.
Ошибка при аугментации 20240120_162912_jpg.rf.29699b9c2c3bc8888f76923fb517f311.jpg: Expected y_max for bbox [0.7616979 0.9906417 0.8364572 1.0000601 0.       ] to be in the range [0.0, 1.0], got 1.0000600814819336.


 22%|██▏       | 169/754 [00:17<01:00,  9.72it/s]

...

Ошибка при аугментации DJI_0276_JPG.rf.9ab500a2b99c5e877759996d7f0a7777.jpg: Expected y_max for bbox [0.24185605 0.9754209  0.2730341  1.0000504  0.        ] to be in the range [0.0, 1.0], got 1.00005042552948.
Ошибка при аугментации DJI_0276_JPG.rf.9ab500a2b99c5e877759996d7f0a7777.jpg: Expected y_max for bbox [0.24185605 0.9754209  0.2730341  1.0000504  0.        ] to be in the range [0.0, 1.0], got 1.00005042552948.
Ошибка при аугментации DJI_0276_JPG.rf.9ab500a2b99c5e877759996d7f0a7777.jpg: Expected y_max for bbox [0.24185605 0.9754209  0.2730341  1.0000504  0.        ] to be in the range [0.0, 1.0], got 1.00005042552948.
Ошибка при аугментации DJI_0276_JPG.rf.9ab500a2b99c5e877759996d7f0a7777.jpg: Expected y_max for bbox [0.24185605 0.9754209  0.2730341  1.0000504  0.        ] to be in the range [0.0, 1.0], got 1.00005042552948.


 97%|█████████▋| 728/754 [01:46<00:04,  5.35it/s]

Ошибка при аугментации DJI_0181_JPG.rf.4ce43776caf50c48ad56c02fef6b3005.jpg: Expected y_max for bbox [0.5299242  0.97946125 0.5345549  1.0000639  0.        ] to be in the range [0.0, 1.0], got 1.0000638961791992.
Ошибка при аугментации DJI_0181_JPG.rf.4ce43776caf50c48ad56c02fef6b3005.jpg: Expected y_max for bbox [0.5299242  0.97946125 0.5345549  1.0000639  0.        ] to be in the range [0.0, 1.0], got 1.0000638961791992.
Ошибка при аугментации DJI_0181_JPG.rf.4ce43776caf50c48ad56c02fef6b3005.jpg: Expected y_max for bbox [0.5299242  0.97946125 0.5345549  1.0000639  0.        ] to be in the range [0.0, 1.0], got 1.0000638961791992.
Ошибка при аугментации DJI_0181_JPG.rf.4ce43776caf50c48ad56c02fef6b3005.jpg: Expected y_max for bbox [0.5299242  0.97946125 0.5345549  1.0000639  0.        ] to be in the range [0.0, 1.0], got 1.0000638961791992.


100%|██████████| 754/754 [01:51<00:00,  6.79it/s]


Ошибка при аугментации DJI_0055_JPG.rf.6790bfee2af07b343b52c463b1084cb7.jpg: Expected y_max for bbox [0.2534091  0.993266   0.25737688 1.0000203  0.        ] to be in the range [0.0, 1.0], got 1.0000202655792236.
Ошибка при аугментации DJI_0055_JPG.rf.6790bfee2af07b343b52c463b1084cb7.jpg: Expected y_max for bbox [0.2534091  0.993266   0.25737688 1.0000203  0.        ] to be in the range [0.0, 1.0], got 1.0000202655792236.
Ошибка при аугментации DJI_0055_JPG.rf.6790bfee2af07b343b52c463b1084cb7.jpg: Expected y_max for bbox [0.2534091  0.993266   0.25737688 1.0000203  0.        ] to be in the range [0.0, 1.0], got 1.0000202655792236.
Ошибка при аугментации DJI_0055_JPG.rf.6790bfee2af07b343b52c463b1084cb7.jpg: Expected y_max for bbox [0.2534091  0.993266   0.25737688 1.0000203  0.        ] to be in the range [0.0, 1.0], got 1.0000202655792236.
Оригинальный датасет: 754 изображений
Аугментированный датасет: 3658 изображений

Пример аугментированных данных:
                                  

In [10]:
#  данные после аугментации
category_counts = {
    "corrosion": 0,
    "lightning": 0,
    "lightning receptor": 0,
    "missing teeth": 0,
    "patch": 0
}

for obj_list in augmented_df['objects']:
    objects = ast.literal_eval(obj_list)

    for obj in objects:
        category_name = obj[0]
        if category_name in category_counts:
            category_counts[category_name] += 1

# Выводим результаты
for category_id, category_name in zip(range(1, len(category_counts) + 1), category_counts.keys()):
    print(f"ID: {category_id}, Название: {category_name}, Количество: {category_counts[category_name]}")

ID: 1, Название: corrosion, Количество: 33681
ID: 2, Название: lightning, Количество: 147
ID: 3, Название: lightning receptor, Количество: 441
ID: 4, Название: missing teeth, Количество: 7928
ID: 5, Название: patch, Количество: 1199


In [13]:
!cp -r "/content/drive/MyDrive/CNN/Wind Turbine damage/yolo_wind_turbine_backup" "/content/"

cp: cannot stat '/content/drive/MyDrive/CNN/Wind Turbine damage/yolo_wind_turbine_backup': No such file or directory


In [ ]:
augmented_df = pd.read_csv('/content/drive/MyDrive/Wind Turbine damage/ augmented_df_full.csv')
df_valid = pd.read_csv('/content/drive/MyDrive/Wind Turbine damage/ df_valid.csv')

In [12]:
print("Hello, World!")

Hello, World!


In [11]:
mkdir '/kaggle/working/yolo_wind'

In [12]:
base_path = '/kaggle/working/yolo_wind'

In [13]:
# Создаем YAML файл конфигурации
yaml_content = {
    'path': base_path,
    'train': 'images/train',
    'val': 'images/val',
    'names': ['corrosion', 'lightning', 'lightning receptor', 'missing teeth', 'patch']
}

with open(os.path.join(base_path, 'wind_turbine.yaml'), 'w') as f:
    yaml.dump(yaml_content, f)

print("Подготовка данных завершена!")

Подготовка данных завершена!


In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


In [15]:
! pip install ultralytics

In [16]:
import os
from pathlib import Path
import shutil
import json
from tqdm import tqdm

# Пути с учётом пробелов
coco_train_path = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train/_annotations.coco.json'
coco_val_path = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/valid/_annotations.coco.json'
yolo_base_path = '/kaggle/working/yolo_wind'
images_train_path = os.path.join(yolo_base_path, 'images/train')
images_val_path = os.path.join(yolo_base_path, 'images/val')
labels_train_path = os.path.join(yolo_base_path, 'labels/train')
labels_val_path = os.path.join(yolo_base_path, 'labels/val')
source_train_path = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train'
source_val_path = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/valid'

# Создание папок
Path(images_train_path).mkdir(parents=True, exist_ok=True)
Path(images_val_path).mkdir(parents=True, exist_ok=True)
Path(labels_train_path).mkdir(parents=True, exist_ok=True)
Path(labels_val_path).mkdir(parents=True, exist_ok=True)

# Классы
class_names = ['corrosion', 'lightning', 'lightning receptor', 'missing teeth', 'patch']
class_map = {name: idx for idx, name in enumerate(class_names)}


def coco_to_yolo(coco_json_path, images_dir, labels_dir, source_dir):
    if not os.path.exists(coco_json_path):
        print(f"Файл {coco_json_path} не найден!")
        return

    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    image_id_to_filename = {img['id']: img['file_name'] for img in coco_data['images']}
    image_id_to_size = {img['id']: (img['width'], img['height']) for img in coco_data['images']}
    annotations = coco_data['annotations']
    categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
    images_copied = 0
    labels_created = 0

    for ann in tqdm(annotations, desc=f"Processing annotations from {coco_json_path}"):
        image_id = ann['image_id']
        filename = image_id_to_filename.get(image_id)
        width, height = image_id_to_size.get(image_id)

        if not filename:
            print(f"Изображение с ID {image_id} не найдено в аннотациях!")
            continue

        bbox = ann['bbox']
        x_min, y_min, width_bbox, height_bbox = bbox  # Переименовано для ясности

        # Проверка и нормализация координат
        x_center = (x_min + width_bbox / 2) / width
        y_center = (y_min + height_bbox / 2) / height
        normalized_width = width_bbox / width
        normalized_height = height_bbox / height

        # Проверка, что координаты находятся в диапазоне [0, 1]
        if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 <= normalized_width <= 1 and 0 <= normalized_height <= 1):
            print(f"Обнаружены аномальные координаты для {filename}: x_center={x_center}, y_center={y_center}, width={normalized_width}, height={normalized_height}")
            continue  # Пропустить эту аннотацию

        category_id = ann['category_id']
        class_id = class_map.get(categories.get(category_id), -1)
        if class_id == -1:
            print(f"Категория с ID {category_id} не найдена в class_map!")
            continue

        src_img_path = os.path.join(source_dir, filename)
        if os.path.exists(src_img_path):
            dst_img_path = os.path.join(images_dir, filename)
            Path(dst_img_path).parent.mkdir(parents=True, exist_ok=True)
            shutil.copy(src_img_path, dst_img_path)
            images_copied += 1
        else:
            print(f"Изображение {filename} не найдено в {source_dir}!")

        label_filename = os.path.splitext(filename)[0] + '.txt'
        label_path = os.path.join(labels_dir, label_filename)
        with open(label_path, 'a') as f:
            f.write(f"{class_id} {x_center} {y_center} {normalized_width} {normalized_height}\n")
            labels_created += 1

    print(f"Скопировано {images_copied} изображений, создано {labels_created} аннотаций в {labels_dir}")


# Выполнение конвертации
coco_to_yolo(coco_train_path, images_train_path, labels_train_path, source_train_path)
coco_to_yolo(coco_val_path, images_val_path, labels_val_path, source_val_path)

print("Конвертация завершена. Проверьте папки images и labels.")

Processing annotations from /kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train/_annotations.coco.json: 100%|██████████| 9150/9150 [00:35<00:00, 258.67it/s]


Скопировано 9150 изображений, создано 9150 аннотаций в /kaggle/working/yolo_wind/labels/train


Processing annotations from /kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/valid/_annotations.coco.json: 100%|██████████| 2619/2619 [00:13<00:00, 194.51it/s]

Скопировано 2619 изображений, создано 2619 аннотаций в /kaggle/working/yolo_wind/labels/val
Конвертация завершена. Проверьте папки images и labels.


In [17]:
 from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [30]:
import os
from ultralytics import YOLO
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from torch.utils.data import Dataset, DataLoader
import cv2
import pandas as pd
import ast
import numpy as np

# 1. Подготовка данных для PyTorch Lightning

class CustomDataset(Dataset):
    def __init__(self, dataframe, images_dir, transform=None):
        self.dataframe = dataframe
        self.images_dir = images_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_path = os.path.join(self.images_dir, row['image_id'])
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        objects_str = row['objects']
        objects = ast.literal_eval(objects_str)
        bboxes = [obj[1:] for obj in objects]  # COCO формат: x, y, width, height
        labels = [class_names.index(obj[0]) for obj in objects]

        if self.transform:
            transformed = self.transform(image=image, bboxes=bboxes, class_labels=labels)
            image = transformed['image']
            bboxes = transformed['bboxes']
            labels = transformed['class_labels']

        # YOLO формат: class_id, x_center, y_center, width, height (нормализованные)
        h, w, _ = image.shape
        yolo_bboxes = []
        for bbox in bboxes:
            x, y, bbox_w, bbox_h = bbox
            x_center = (x + bbox_w / 2) / w
            y_center = (y + bbox_h / 2) / h
            norm_w = bbox_w / w
            norm_h = bbox_h / h
            yolo_bboxes.append([x_center, y_center, norm_w, norm_h])

        yolo_bboxes = torch.tensor(yolo_bboxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        image = torch.tensor(image).permute(2, 0, 1).float() / 255.0  # нормализация и изменение размерности
        return image, yolo_bboxes, labels

# 2. PyTorch Lightning Module
class YOLOModule(pl.LightningModule):
    def __init__(self, model, num_classes, lr=1e-3):
        super().__init__()
        self.model = model
        self.num_classes = num_classes
        self.lr = lr

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, boxes, labels = batch
        outputs = self.model(images)

        # Вычисление loss
        loss = 0  # Заменить на вашу функцию потерь
        return loss

    def validation_step(self, batch, batch_idx):
        images, boxes, labels = batch
        outputs = self.model(images)
        val_loss = 0  # Заменить на вашу функцию потерь
        return val_loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr)
        return optimizer


# 3. Подготовка данных
train_df = pd.read_csv('/kaggle/working/augmented_annotations.csv')
val_df = pd.read_csv('/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/valid/_annotations.coco.json')

images_folder = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train'  # Путь к изображениям

train_dataset = CustomDataset(train_df, images_folder, transform=transform)
val_dataset = CustomDataset(val_df, images_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# 4. Инициализация модели и модуля Lightning
model = YOLO("yolov8n.pt").model
module = YOLOModule(model, num_classes=5, lr=1e-3)

# 5. Early Stopping
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=10,
    mode='min'
)

# 6. Trainer
trainer = pl.Trainer(
    max_epochs=50,
    accelerator="gpu",
    devices=1,
    callbacks=[early_stopping_callback]
)

# 7. Training
trainer.fit(module, train_loader, val_loader)

E0000 00:00:1750029822.083883      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750029822.134492      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/utilities/data.py:106: Total length of `DataLoader` across ranks is zero. Please make sure this was your intention.
[ WARN:0@4415.797] global loadsave.cpp:268 findDecoder imread_('/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train/aug3_20240120_164348_mp4-4112_jpg.rf.98a4b89018d1a402534561ea83a063e2.jpg'): can't open/read file: check file path/integrity
[ WARN:0@4415.798] global loadsave.cpp:268 findDecoder imread_('/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train/aug0_20240120_164348_mp4-2302_jpg.rf.f2ce05787404ce39a8faff99c2257ce8.jpg'): can't open/read file: check file path/integrity


Training: |          | 0/? [00:00<?, ?it/s]

[ WARN:0@4415.837] global loadsave.cpp:268 findDecoder imread_('/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train/aug2_20240120_164348_mp4-4122_jpg.rf.3f652aea7d079be4dd2ca5050b8e1ac6.jpg'): can't open/read file: check file path/integrity
[ WARN:0@4415.847] global loadsave.cpp:268 findDecoder imread_('/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train/aug3_DJI_0229_JPG.rf.83130bc27eba441a50cc2b0f4083658e.jpg'): can't open/read file: check file path/integrity
[ WARN:0@4415.850] global loadsave.cpp:268 findDecoder imread_('/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train/aug2_20240120_161132_jpg.rf.16a357f2ba96a0fc4599d213af9f6272.jpg'): can't open/read file: check file path/integrity
[ WARN:0@4416.137] global loadsave.cpp:268 findDecoder imread_('/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train/aug0_DJI_0282_JPG.rf.ead995ea7ffb32e7e84ae430cefbfebe.jpg'): can't open/read file: check file path/integrity


error: Caught error in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/tmp/ipykernel_35/812389737.py", line 27, in __getitem__
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
cv2.error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion 

# Предсказание

In [19]:
import os
import cv2
import pandas as pd
import json
from ultralytics import YOLO
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [20]:
best_model = YOLO(os.path.join('runs', 'detect', 'train', 'weights', 'best.pt'))

In [21]:

test_folder = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/test'

results = []

for img_name in os.listdir(test_folder):
    if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    img_path = os.path.join(test_folder, img_name)
    img = cv2.imread(img_path)
    if img is None:
        continue

    predictions = best_model.predict(img, conf=0.5)

    img_objects = []
    for pred in predictions:
        for box in pred.boxes:
            class_id = int(box.cls)
            class_name = best_model.names[class_id]
            conf = float(box.conf)
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            w = x2 - x1
            h = y2 - y1

            img_objects.append([class_name, x1, y1, w, h, conf])

    results.append({
        "image_id": img_name,
        "objects": img_objects
    })

df_results = pd.DataFrame(results)


0: 640x640 (no detections), 7.2ms
Speed: 3.8ms preprocess, 7.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 36.4ms
Speed: 2.9ms preprocess, 36.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.0ms
Speed: 2.9ms preprocess, 7.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 640x384 (no detections), 35.0ms
Speed: 2.5ms preprocess, 35.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 384x640 (no detections), 9.8ms
Speed: 3.0ms preprocess, 9.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 640x640 1 missing teeth, 7.1ms
Speed: 3.6ms preprocess, 7.1ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 6.5ms
Speed: 3.7ms preprocess, 6.5ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x384 (no detections), 7.8ms
Speed: 2.1ms preprocess, 7.8ms inference, 0.

In [32]:
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    xA = max(x1, x2)
    yA = max(y1, y2)
    xB = min(x1 + w1, x2 + w2)
    yB = min(y1 + h1, y2 + h2)
    inter_area = max(0, xB - xA) * max(0, yB - yA)
    box1_area = w1 * h1
    box2_area = w2 * h2
    iou = inter_area / float(box1_area + box2_area - inter_area)
    return iou

# Пример: сравнение с известными аннотациями
ground_truth = [
    {"image_id": "image1.jpg", "objects": [["corrosion", 100, 150, 50, 30], ["missing teeth", 200, 250, 40, 20]]},
    {"image_id": "image2.jpg", "objects": [["lightning", 300, 400, 60, 25]]}
]
tp, fp, fn = 0, 0, 0
iou_threshold = 0.5

for pred_img in results:
    pred_boxes = pred_img['objects']
    gt_boxes = [gt for gt in ground_truth if gt['image_id'] == pred_img['image_id']]
    for pred_box in pred_boxes:
        matched = False
        for gt_box in gt_boxes:
            iou = calculate_iou(pred_box[1:5], gt_box[1:5])
            if iou > iou_threshold:
                tp += 1
                matched = True
                break
        if not matched:
            fp += 1
    fn += len(gt_boxes) - len([b for b in gt_boxes if any(calculate_iou(b[1:5], p[1:5]) > iou_threshold for p in pred_boxes)])

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
print(f"Precision: {precision}, Recall: {recall}")

Precision: 0.0, Recall: 0


In [22]:
df_results

,image_id,objects
0,20240120_160706_jpg.rf.6b2e2b4a13dc2e6d5946abc...,[]
1,DJI_0706-1-_JPG.rf.266efc3b31622fd7c6e2043e778...,[]
2,DJI_0689-1-_JPG.rf.fa20919f9ea6451ed2d49be4f6f...,[]
3,20240120_164348_mp4-4190_jpg.rf.35c7cf908606d6...,[]
4,DJI_0728-1-_JPG.rf.55ae4b479b0e282856186143e1f...,[]
...,...,...
146,DJI_0755-1-_JPG.rf.5cc110a79566dd9ab3ab412e985...,[]
147,20240120_155211_mp4-0015_jpg.rf.62ada8738a17b0...,[]
148,20240120_160557_jpg.rf.4d9909256e4bcb526d3eb1b...,"[[corrosion, 70.9300537109375, 2414.7143554687..."
149,20240120_160457-0-_jpg.rf.609b8e725f177836e00f...,"[[corrosion, 1502.375732421875, 1117.431762695..."


In [23]:
df_results.to_csv('petukhov_output.csv')
# df_results.to_csv("petukhov_output_2.csv", index=False, sep=",", quoting=1)

In [35]:
df_results[8:]

,image_id,objects
8,20240120_155211_mp4-0086_jpg.rf.1d43d0c9b738b9...,[]
9,20240120_161103-0-_jpg.rf.7ac8bc94eb0190afc02b...,"[[corrosion, 2111.861083984375, 2808.032714843..."
10,20240120_162905-0-_jpg.rf.cec91a04d4d97418ef53...,[]
11,20240120_155211_mp4-0075_jpg.rf.82235fee115b78...,[]
12,20240120_155211_mp4-0110_jpg.rf.a5ed327b0bb16f...,[]
...,...,...
146,DJI_0980_JPG.rf.9aae1abc3fdead93968f7a06073e9b...,[]
147,DJI_0997_JPG.rf.9f5f2b599b0927db0d1121a39516f4...,"[[corrosion, 2946.65966796875, 807.39801025390..."
148,DJI_0995_JPG.rf.09f70312420808da47c3a4aaf186ca...,[]
149,DJI_0993_JPG.rf.f33b6663ee70d58a17f1b41a031a28...,[]


In [36]:
pd.read_csv('sample_submission_with_errors.csv')

,image_id,objects
0,20240120_153843-0-_jpg.rf.60b55d7dd594d0869bf7...,"[[""corrosion"", 45.85849989276216, 96.317499685..."
1,20240120_153843_jpg.rf.b24d413669fa57588901891...,[]
2,20240120_153658_jpg.rf.66dc04dc4ee8194ccc19397...,"[[""damage"", 152.5677603675233, 134.65200109050..."
3,DJI_0765-1-_JPG.rf.500d5a743566278e0be2d7cd60c...,[]
4,DJI_0987_JPG.rf.c2002d99afd301f941c19bd6e56a91...,"[[""patch"", 2600.8498398242286, 1073.0071925444..."
...,...,...
146,20240120_163650-0-_jpg.rf.6ff26d6832686600c6f1...,"[[""damage"", 954.531026827425, 1788.35393111455..."
147,20240120_163730-0-_jpg.rf.7139d077e629ae78b030...,"[[""missing teeth"", 642.4468737245788, 268.7765..."
148,20240120_155211_mp4-0126_jpg.rf.5a9e67fd5bce79...,"[[""corrosion"", 82.17908120401553, 160.38408485..."
149,20240120_155211_mp4-0084_jpg.rf.de984895faf812...,[]


In [24]:
pd.read_csv('petukhov_output.csv')

,Unnamed: 0,image_id,objects
0,0,20240120_160706_jpg.rf.6b2e2b4a13dc2e6d5946abc...,[]
1,1,DJI_0706-1-_JPG.rf.266efc3b31622fd7c6e2043e778...,[]
2,2,DJI_0689-1-_JPG.rf.fa20919f9ea6451ed2d49be4f6f...,[]
3,3,20240120_164348_mp4-4190_jpg.rf.35c7cf908606d6...,[]
4,4,DJI_0728-1-_JPG.rf.55ae4b479b0e282856186143e1f...,[]
...,...,...,...
146,146,DJI_0755-1-_JPG.rf.5cc110a79566dd9ab3ab412e985...,[]
147,147,20240120_155211_mp4-0015_jpg.rf.62ada8738a17b0...,[]
148,148,20240120_160557_jpg.rf.4d9909256e4bcb526d3eb1b...,"[['corrosion', 70.9300537109375, 2414.71435546..."
149,149,20240120_160457-0-_jpg.rf.609b8e725f177836e00f...,"[['corrosion', 1502.375732421875, 1117.4317626..."


In [25]:
df_user = pd.read_csv('petukhov_output.csv')

In [26]:
df_user['objects'] = df_user['objects'].apply(lambda x: x.replace("'", '"'))

In [27]:
del df_user['Unnamed: 0']

In [28]:
df_user.to_csv('aapetukhov_output_3.csv')